## Preprocessing

In [8]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

URLError: <urlopen error [Errno 54] Connection reset by peer>

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns = ["EIN", "NAME"], inplace = True)

In [9]:
# Determine the number of unique values in each column.
application_df.nunique()

EIN                       34299
NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [10]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [11]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
atemp = application_df["APPLICATION_TYPE"].value_counts()

application_types_to_replace = atemp[atemp < 500].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [12]:
# Look at CLASSIFICATION value counts for binning
application_df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2170        1
C1370        1
C2380        1
C1900        1
C1248        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [13]:
# You may find it helpful to look at CLASSIFICATION value counts >1
ctemp = application_df.CLASSIFICATION.value_counts()
ctemp[ctemp > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
C1237        9
C1235        9
C7210        7
C1720        6
C4100        6
C2400        6
C1600        5
C1257        5
C1260        3
C0           3
C2710        3
C1234        2
C1256        2
C1246        2
C3200        2
C1267        2
Name: CLASSIFICATION, dtype: int64

In [14]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = ctemp[ctemp < 1000].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [15]:
application_df["SPECIAL_CONSIDERATIONS"]

0        N
1        N
2        N
3        N
4        N
        ..
34294    N
34295    N
34296    N
34297    N
34298    N
Name: SPECIAL_CONSIDERATIONS, Length: 34299, dtype: object

In [16]:
# Convert categorical data to numeric with `pd.get_dummies`

# Code binary column
application_df["SPECIAL_CONSIDERATIONS"] = [0 if x == "N" else 1 for x in application_df["SPECIAL_CONSIDERATIONS"]]

dummy_df = pd.get_dummies(application_df)

In [17]:
len(dummy_df.loc[:,(dummy_df.columns != "IS_SUCCESSFUL")].columns)

len(dummy_df.columns)

19612

In [18]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
xset, yset = dummy_df.loc[:,(dummy_df.columns != "IS_SUCCESSFUL")], dummy_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
xtrain, xtest, ytrain, ytest = train_test_split(xset, yset)

In [19]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(xtrain)

# Scale the data
xtrain_scaled = X_scaler.transform(xtrain)
xtest_scaled = X_scaler.transform(xtest)

## Compile, Train and Evaluate the Model

In [20]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
n_inputs = len(xtrain_scaled[0])

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units = (2 * n_inputs), input_dim = n_inputs, activation = "relu"))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units = 20, activation = "sigmoid"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units = 1, input_dim = n_inputs, activation = "sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 39222)             769221864 
_________________________________________________________________
dense_1 (Dense)              (None, 20)                784460    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 21        
Total params: 770,006,345
Trainable params: 770,006,345
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Compile the model
nn_model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [22]:
# Train the model
fit_model = nn_model.fit(xtrain_scaled, ytrain, epochs = 100)

2023-05-12 13:12:47.973564: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/100
201/804 [======>.......................] - ETA: 24:02 - loss: 0.6864 - accuracy: 0.6987

KeyboardInterrupt: 

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(xtest_scaled,ytest,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5613 - accuracy: 0.7275 - 509ms/epoch - 2ms/step
Loss: 0.5612988471984863, Accuracy: 0.7274635434150696


In [ ]:
# Export our model to HDF5 file
nn_model.save(filepath = "AlphabetSoupCharity.h5", save_format = "h5")

In [ ]:
# # This code was done on google colab–– download the HDF5 file with relevant library
# from google.colab import files

# files.download("AlphabetSoupCharity.h5")

##  Model optimization

* Instead of 3 optimization attempts, this code will attempt to optimize with keras_tuner

In [2]:
#   Additional Depedency
import keras_tuner as kt

In [5]:
#   Create method that creates a new Sequential model with hyperparameter options

def create_model(hp):
  nn_model = tf.keras.models.Sequential()

  # Assign input dim per features
  n_inputs = len(xtrain_scaled[0])

  #   Allow keras_tuner to decide which activation function to use in hidden layers
  activation = hp.Choice("activation", ["relu", "sigmoid", "tanh"])

  #   Allow keras_tuner to decide the number of neurons in the first layer
  nn_model.add(tf.keras.layers.Dense(units = hp.Int("first_units",
                                                    min_value = 1,
                                                    max_value = 30,
                                                    step = 2),
                                                activation = activation, 
                                                input_dim = n_inputs))
  
  # Allow keras_tuner to decide the number of hidden layers and neurons in hidden layers
  for i in range(hp.Int("num_layers", 1, 5)):
    nn_model.add(tf.keras.layers.Dense(units = hp.Int("units_" + str(i),
                                                      min_value = 1,
                                                      max_value = 30,
                                                      step = 5),
                                                      activation = activation))

  nn_model.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

  # Compile the model
  nn_model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

  return nn_model

In [7]:
#   Tuner

tuner = kt.Hyperband(
    create_model,
    objective = "val_accuracy",
    max_epochs = 50,
    hyperband_iterations = 3
)

NameError: name 'xtrain_scaled' is not defined